In [2]:
import pandas as pd

ufo_url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(ufo_url)

print("Columns:", list(df.columns))
print("\nSample rows:")
display(df.head(3))


Columns: ['10/10/1949 20:30', 'san marcos', 'tx', 'us', 'cylinder', '2700', '45 minutes', 'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit', '4/27/2004', '29.8830556', '-97.9411111']

Sample rows:


,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit,4/27/2004,29.8830556,-97.9411111
0,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
1,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
2,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833


In [3]:
import pandas as pd, numpy as np, altair as alt

ufo_url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
cols = ['datetime','city','state','country','shape','duration_seconds','duration','comments','date_posted','latitude','longitude']
df = pd.read_csv(ufo_url, header=None, names=cols)
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df['month'] = df['datetime'].dt.to_period('M').dt.to_timestamp()
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
print(df.shape)
display(df.head(3))


(80332, 12)


,datetime,city,state,country,shape,duration_seconds,duration,comments,date_posted,latitude,longitude,month
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949-10-01
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949-10-01
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955-10-01


In [6]:
import altair as alt, re

df_us = df[df['state'].apply(lambda x: bool(re.match(r'^[A-Za-z]{2}$', str(x))))]

m = df.groupby('month', as_index=False).size()
sel = alt.selection_interval(encodings=['x'])

top_states = (
    df_us.groupby('state', as_index=False)
         .size()
         .sort_values('size', ascending=False)
         .head(20)['state']
)

upper = (
    alt.Chart(m)
      .mark_area(opacity=0.6)
      .encode(x='month:T', y='size:Q', tooltip=['month:T','size:Q'])
      .add_params(sel)
      .properties(width='container', height=150, title='UFO sightings per month')
)

lower = (
    alt.Chart(df_us[df_us['state'].isin(top_states)])
      .mark_bar()
      .encode(x=alt.X('state:N', sort='-y', title='State'),
              y=alt.Y('count():Q', title='Sightings'))
      .transform_filter(sel)
      .properties(width='container', height=260, title='Top 20 states (filtered by time)')
)

(upper & lower).save('docs/charts/ufo_time_state.html')
print("Chart 1 -> docs/charts/ufo_time_state.html")


Chart 1 -> docs/charts/ufo_time_state.html


In [8]:
import altair as alt

opts = ['All'] + sorted(df['shape'].dropna().unique().tolist())
pick = alt.selection_point(fields=['shape'], bind=alt.binding_select(options=opts, name='Shape: '), value='All')

viz_b = (
    alt.Chart(df.dropna(subset=['latitude','longitude']))
      .mark_circle(size=26, opacity=0.55)
      .encode(
          x='longitude:Q', y='latitude:Q',
          color=alt.condition(alt.datum.shape == pick, alt.value('steelblue'), alt.value('#cfcfcf')),
          tooltip=['datetime:T','city:N','state:N','country:N','shape:N']
      )
      .add_params(pick)
      .properties(width='container', height=480, title='UFO sightings — lat/long with shape highlight')
)

viz_b.save('docs/charts/ufo_map_shape.html')
print('Chart 2 -> docs/charts/ufo_map_shape.html')


Chart 2 -> docs/charts/ufo_map_shape.html
